# Todos:
1) importar test set
2) ordenar AR-GARCHs según su capacidad de predicción en el train set 
   1) seguro distintas métricas nos dan distintos modelos. por lo tanto, 
3) guardar distintos residuos para comparar contra los tres tipos de modelos

## Startup

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import arch

import os
import pickle

In [4]:
np.random.seed(42)

In [5]:
dataroute=os.path.join("..",  "data")
resultsroute=os.path.join("..",  "results")

## Data Retrieval

In [6]:
start='2013-01-01'
end="2023-06-01"

name=f'processed_dataset_{start}_{end}.pickle'
filename=os.path.join(dataroute, name)

with open(filename, 'rb') as handle:
    data=pickle.load(handle)

## GARCH Training

In [7]:
# Define the range of p and q values
p_values = [1, 2, 3]  # Example: p values
q_values = [0, 1, 2, 3]  # Example: q values
# all models with q=0 are exclusively ARCH (non-GARCH)

In [8]:
models = {}
predict = {}

In [13]:
for key, ohlc_df in data.items():
    if ohlc_df["log_rets"].isna().any():
        print(key)
        #display(ohlc_df.loc[ohlc_df["log_rets"].isna()])

USD_GGAL
USD_YPF
USD_EDN
USD_BMA
USD
USD_^MERV


In [9]:
# Estimate ARMA-ARCH and ARMA-GARCH models for different p and q values
for key, ohlc_df in data.items():
    returns = ohlc_df['log_rets']
    
    models[key] = {}
    predict[key] = {}

    for p in p_values:
        for q in q_values:
            for dist in ['Normal', 'StudentsT']:
                print(p+q)
                model = arch.arch_model(returns, 
                                        mean="AR",
                                        lags=1,
                                        vol='Garch', 
                                        p=p, q=q, dist=dist, 
                                        rescale=False)
                results = model.fit()

                models[key][(p, q, dist)] = results

1
Iteration:      1,   Func. Count:      6,   Neg. LLF: 10848533890.952875
Iteration:      2,   Func. Count:     18,   Neg. LLF: 5249402.08276884
Iteration:      3,   Func. Count:     26,   Neg. LLF: 88572.35058949565
Iteration:      4,   Func. Count:     34,   Neg. LLF: 9497.100331940472
Iteration:      5,   Func. Count:     42,   Neg. LLF: 9942.931310347381
Iteration:      6,   Func. Count:     50,   Neg. LLF: 11313090856.939453
Iteration:      7,   Func. Count:     57,   Neg. LLF: 10265.33956489234
Iteration:      8,   Func. Count:     65,   Neg. LLF: -2234.503648570537
Iteration:      9,   Func. Count:     71,   Neg. LLF: -2205.2209724671998
Iteration:     10,   Func. Count:     77,   Neg. LLF: -2236.544534411522
Iteration:     11,   Func. Count:     83,   Neg. LLF: -953.9173223296295
Iteration:     12,   Func. Count:     90,   Neg. LLF: -2230.615981165848
Iteration:     13,   Func. Count:     96,   Neg. LLF: -3302.8127028356007
Iteration:     14,   Func. Count:    102,   Neg. LLF:

c:\Users\alfredo.sampron\AppData\Local\miniconda3\envs\hmm\Lib\site-packages\arch\univariate\base.py:765: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:      4,   Func. Count:     58,   Neg. LLF: 11623.160155901935
Iteration:      5,   Func. Count:     68,   Neg. LLF: 10254.251323341166
Iteration:      6,   Func. Count:     76,   Neg. LLF: 88641.98595850512
Iteration:      7,   Func. Count:     86,   Neg. LLF: 10612.555731826758
Iteration:      8,   Func. Count:     95,   Neg. LLF: 14421.634518125507
Iteration:      9,   Func. Count:    105,   Neg. LLF: 18042.279426682155
Iteration:     10,   Func. Count:    114,   Neg. LLF: 8635.682919866513
Iteration:     11,   Func. Count:    123,   Neg. LLF: 19974.939094928566
Iteration:     12,   Func. Count:    132,   Neg. LLF: 8116.662319717041
Iteration:     13,   Func. Count:    140,   Neg. LLF: 7834.87401818172
Iteration:     14,   Func. Count:    154,   Neg. LLF: 8693.377031082844
Iteration:     15,   Func. Count:    163,   Neg. LLF: 10776.110390534854
Iteration:     16,   Func. Count:    172,   Neg. LLF: 12974.25429886141
Iteration:     17,   Func. Count:    181,   Neg. LLF: 3680

c:\Users\alfredo.sampron\AppData\Local\miniconda3\envs\hmm\Lib\site-packages\arch\univariate\base.py:765: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:     44,   Func. Count:    572,   Neg. LLF: 30632.22863560078
Iteration:     45,   Func. Count:    582,   Neg. LLF: 30608.276850500144
Iteration:     46,   Func. Count:    592,   Neg. LLF: 30599.547224057187
Iteration:     47,   Func. Count:    602,   Neg. LLF: 30586.308149615397
Iteration:     48,   Func. Count:    612,   Neg. LLF: 30584.844605111422
Iteration:     49,   Func. Count:    622,   Neg. LLF: 30583.322959297097
Iteration:     50,   Func. Count:    632,   Neg. LLF: 30573.518746080375
Iteration:     51,   Func. Count:    642,   Neg. LLF: 30518.097103397056
Iteration:     52,   Func. Count:    652,   Neg. LLF: 30214.653213628335
Iteration:     53,   Func. Count:    662,   Neg. LLF: 28355.79218775057
Iteration:     54,   Func. Count:    672,   Neg. LLF: 33191.09533817818
Iteration:     55,   Func. Count:    683,   Neg. LLF: 20151.099685646645
Iteration:     56,   Func. Count:    693,   Neg. LLF: 30497.464520801354
Iteration:     57,   Func. Count:    705,   Neg. LLF: 

...

c:\Users\alfredo.sampron\AppData\Local\miniconda3\envs\hmm\Lib\site-packages\arch\univariate\base.py:765: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:      3,   Func. Count:     44,   Neg. LLF: 4723.781279575718
Iteration:      4,   Func. Count:     55,   Neg. LLF: 19792.776487680887
Iteration:      5,   Func. Count:     67,   Neg. LLF: 4129.147326707469
Iteration:      6,   Func. Count:     79,   Neg. LLF: -2168.179045507459
Iteration:      7,   Func. Count:     89,   Neg. LLF: -1344.1161229617005
Iteration:      8,   Func. Count:    102,   Neg. LLF: -2366.170633926019
Iteration:      9,   Func. Count:    113,   Neg. LLF: -3255.7844169126265
Iteration:     10,   Func. Count:    124,   Neg. LLF: 63212.22726628183
Iteration:     11,   Func. Count:    144,   Neg. LLF: 13489.846125530867
Iteration:     12,   Func. Count:    159,   Neg. LLF: 9219.568391458119
Iteration:     13,   Func. Count:    169,   Neg. LLF: 13864.111326434597
Iteration:     14,   Func. Count:    180,   Neg. LLF: 69141.52407382867
Iteration:     15,   Func. Count:    194,   Neg. LLF: 37968.961232852336
Iteration:     16,   Func. Count:    205,   Neg. LLF: 

c:\Users\alfredo.sampron\AppData\Local\miniconda3\envs\hmm\Lib\site-packages\arch\univariate\base.py:765: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:     24,   Func. Count:    245,   Neg. LLF: -5212.028623151389
Iteration:     25,   Func. Count:    252,   Neg. LLF: -5212.029191259077
Iteration:     26,   Func. Count:    259,   Neg. LLF: -5212.029222890803
Iteration:     27,   Func. Count:    266,   Neg. LLF: -5212.029219128355
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5212.029222890801
            Iterations: 30
            Function evaluations: 276
            Gradient evaluations: 27
3
Iteration:      1,   Func. Count:      8,   Neg. LLF: 161421.5644003993
Iteration:      2,   Func. Count:     20,   Neg. LLF: 110621872.0584157
Iteration:      3,   Func. Count:     35,   Neg. LLF: 200668.17871306086
Iteration:      4,   Func. Count:     48,   Neg. LLF: 73713.66268903177
Iteration:      5,   Func. Count:     57,   Neg. LLF: 76373.08878804679
Iteration:      6,   Func. Count:     68,   Neg. LLF: 10954.982808725747
Iteration:      7,   Func. Count:     78,   Neg. LLF: 38569.8

ValueError: NaN or inf values found in y. y must contains only finite values.

# Model prediction

Function documentation: https://arch.readthedocs.io/en/latest/univariate/generated/generated/arch.univariate.base.ARCHModelResult.forecast.html#arch.univariate.base.ARCHModelResult.forecast

In [2]:
for key, ohlc_df in data.items():
    for p in p_values:
        for q in q_values:
            for dist in ['Normal', 'StudentsT']:
                # Predictions on the training data
                pred = results.forecast()
                predict[key][(p, q, dist)] = predict

NameError: name 'data' is not defined

# Plotting

In [23]:
def plot_close_rets_vol(data, key, comp):
    model=models[f"{key}_{comp}_model"]
    prediction=models[f"{key}_{comp}_prediction"]
    states=set(prediction)

    fig=plt.figure(figsize = (20, 20))
    plt.tight_layout()
    plt.title(f"{key} Close, Log returns and intraday Vol\n{comp} states")

    for subplot, var in zip(range(1,4), ["Close", "log_rets", "gk_vol"]):    
        plt.subplot(3,1,subplot)
        for i in set(prediction):
            state = (prediction == i)
            x = data[key].index[state]
            y = data[key][var].iloc[state]
            plt.plot(x, y, '.')
        plt.legend(states, fontsize=16)
        
        plt.grid(True)
        plt.xlabel("datetime", fontsize=16)
        plt.ylabel(var, fontsize=16)
            
    plt.savefig(os.path.join(resultsroute, "graphs", 
                             f"{comp}_states", 
                             f"{key}_model_{comp}.png"))

In [ ]:
for key in data.keys():
    for comp in comps:
        plot_close_rets_vol(data, key, comp)
plt.show()